In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from IPython.core.display import display, HTML
from IPython.lib.pretty import pprint
from itertools import chain

In [3]:
from mockdown.view import View

In [4]:
sample_view = View("p", (0, 0, 100, 100), children=[
    View("a1", (10, 10, 45, 90), children=[
        View("b11", (20, 20, 35, 45)),
        View("b12", (20, 55, 35, 80))
    ]),
    View("a2", (55, 10, 90, 90), children=[
        View("b21", (65, 20, 80, 45)),
        View("b22", (65, 55, 80, 80))
    ])
])

In [5]:
html_str = str(sample_view.to_display_html(scale=2))
display(HTML(html_str))

In [6]:
from mockdown.visibility import visible_pairs

edge_pairs = visible_pairs(sample_view, deep=True)
anchor_pairs = [(e1.anchor, e2.anchor) for (e1, e2) in edge_pairs]

In [7]:
html_str = str(sample_view.to_display_html(edge_pairs, scale=2))
display(HTML(html_str))

In [10]:
from mockdown.logic import constraint_pairs
answer = constraint_pairs(sample_view, anchor_pairs)

print(answer)

[{'V': 'a1', 'A': 'right', 'W': 'a2', 'B': 'left'}, {'V': 'b11', 'A': 'bottom', 'W': 'b12', 'B': 'top'}, {'V': 'b21', 'A': 'bottom', 'W': 'b22', 'B': 'top'}, {'V': 'p', 'A': 'top', 'W': 'a1', 'B': 'top'}, {'V': 'p', 'A': 'left', 'W': 'a1', 'B': 'left'}, {'V': 'p', 'A': 'bottom', 'W': 'a1', 'B': 'bottom'}, {'V': 'p', 'A': 'top', 'W': 'a2', 'B': 'top'}, {'V': 'p', 'A': 'bottom', 'W': 'a2', 'B': 'bottom'}, {'V': 'p', 'A': 'right', 'W': 'a2', 'B': 'right'}, {'V': 'a1', 'A': 'top', 'W': 'b11', 'B': 'top'}, {'V': 'a1', 'A': 'left', 'W': 'b11', 'B': 'left'}, {'V': 'a1', 'A': 'right', 'W': 'b11', 'B': 'right'}, {'V': 'a1', 'A': 'left', 'W': 'b12', 'B': 'left'}, {'V': 'a1', 'A': 'bottom', 'W': 'b12', 'B': 'bottom'}, {'V': 'a1', 'A': 'right', 'W': 'b12', 'B': 'right'}, {'V': 'a2', 'A': 'top', 'W': 'b21', 'B': 'top'}, {'V': 'a2', 'A': 'left', 'W': 'b21', 'B': 'left'}, {'V': 'a2', 'A': 'right', 'W': 'b21', 'B': 'right'}, {'V': 'a2', 'A': 'left', 'W': 'b22', 'B': 'left'}, {'V': 'a2', 'A': 'bottom',